In [1]:
# Load a local copy of the current ODYM branch:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from seaborn.palettes import color_palette
import xlrd
import pylab
from copy import deepcopy
import logging as log
import xlwt
import tqdm
import math
from scipy.stats import norm
from tqdm import tqdm
import time
import matplotlib
mpl_logger = log.getLogger("matplotlib")
mpl_logger.setLevel(log.WARNING)  
# For Ipython Notebook only
### Preamble
# Going to parent path
os.getcwd()
os.chdir("..")
os.chdir("..")

# add ODYM module directory to system path, relative
MainPath = os.path.join(os.getcwd(), 'odym', 'modules')
sys.path.insert(0, MainPath)

# add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.getcwd(), 'odym', 'modules'))

# Specify path to dynamic stock model and to datafile, relative
DataPath = os.path.join( 'docs', 'files')

# Specify path to dynamic stock model and to datafile, absolute
DataPath = os.path.join(os.getcwd(), 'docs', 'Files')

import ODYM_Classes as msc # import the ODYM class file
import ODYM_Functions as msf # import the ODYM function file
import dynamic_stock_model as dsm # import the dynamic stock model library

# Initialize loggin routine
log_verbosity = eval("log.DEBUG")
log_filename = 'LogFileTest.md'
[Mylog, console_log, file_log] = msf.function_logger(log_filename, os.getcwd(),
                                                     log_verbosity, log_verbosity)
Mylog.info('### 1. - Initialize.')

#Read main script parameters
#Load project-specific config file
ProjectSpecs_ConFile = 'ODYM_Config_Vehicle_System.xlsx'
Model_Configfile     = xlrd.open_workbook(os.path.join(DataPath, ProjectSpecs_ConFile))
ScriptConfig         = {'Model Setting': Model_Configfile.sheet_by_name('Config').cell_value(3,3)} # Dictionary with config parameters
Model_Configsheet    = Model_Configfile.sheet_by_name('Setting_' + ScriptConfig['Model Setting'])

Name_Scenario        = Model_Configsheet.cell_value(3,3)
print(Name_Scenario)

#Read control and selection parameters into dictionary
ScriptConfig         = msf.ParseModelControl(Model_Configsheet,ScriptConfig)

Mylog.info('Read and parse config table, including the model index table, from model config sheet.')
IT_Aspects,IT_Description,IT_Dimension,IT_Classification,IT_Selector,IT_IndexLetter,\
PL_Names,PL_Description,PL_Version,PL_IndexStructure,PL_IndexMatch,PL_IndexLayer,\
PrL_Number,PrL_Name,PrL_Comment,PrL_Type,ScriptConfig = msf.ParseConfigFile(Model_Configsheet,ScriptConfig,Mylog)    

class_filename       = 'ODYM_Classifications_Master_Vehicle_System.xlsx'
Classfile            = xlrd.open_workbook(os.path.join(DataPath,class_filename))
Classsheet           = Classfile.sheet_by_name('MAIN_Table')
MasterClassification = msf.ParseClassificationFile_Main(Classsheet,Mylog)


Mylog.info('Define model classifications and select items for model classifications according to information provided by config file.')
ModelClassification  = {} # Dict of model classifications
for m in range(0,len(IT_Aspects)):
    ModelClassification[IT_Aspects[m]] = deepcopy(MasterClassification[IT_Classification[m]])
    EvalString = msf.EvalItemSelectString(IT_Selector[m],len(ModelClassification[IT_Aspects[m]].Items))
    if EvalString.find(':') > -1: # range of items is taken
        RangeStart = int(EvalString[0:EvalString.find(':')])
        RangeStop  = int(EvalString[EvalString.find(':')+1::])
        ModelClassification[IT_Aspects[m]].Items = ModelClassification[IT_Aspects[m]].Items[RangeStart:RangeStop]           
    elif EvalString.find('[') > -1: # selected items are taken
        ModelClassification[IT_Aspects[m]].Items = [ModelClassification[IT_Aspects[m]].Items[i] for i in eval(EvalString)]
    elif EvalString == 'all':
        None
    else:
        Mylog.error('Item select error for aspect ' + IT_Aspects[m] + ' were found in datafile.')
        break

# Define model index table and parameter dictionary
Mylog.info('### 2.2 - Define model index table and parameter dictionary')
Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End   = int(max(ModelClassification['Time'].Items))
Model_Duration   = Model_Time_End - Model_Time_Start + 1

Mylog.info('Define index table dataframe.')
IndexTable = pd.DataFrame({'Aspect'        : IT_Aspects,  # 'Time' and 'Element' must be present!
                           'Description'   : IT_Description,
                           'Dimension'     : IT_Dimension,
                           'Classification': [ModelClassification[Aspect] for Aspect in IT_Aspects],
                           'IndexLetter'   : IT_IndexLetter})  # Unique one letter (upper or lower case) indices to be used later for calculations.

# Default indexing of IndexTable, other indices are produced on the fly
IndexTable.set_index('Aspect', inplace=True)

# Add indexSize to IndexTable:
IndexTable['IndexSize'] = pd.Series([len(IndexTable.Classification[i].Items) for i in range(0, len(IndexTable.IndexLetter))],
                                    index=IndexTable.index)

# list of the classifications used for each indexletter
IndexTable_ClassificationNames = [IndexTable.Classification[i].Name for i in range(0, len(IndexTable.IndexLetter))]


# Define dimension sizes
Nt = len(IndexTable.Classification[IndexTable.index.get_loc('Time')].Items)
Nc = len(IndexTable.Classification[IndexTable.index.get_loc('Age-cohort')].Items)
Ng = len(IndexTable.Classification[IndexTable.index.get_loc('Drive_train')].Items)
Ne = len(IndexTable.Classification[IndexTable.index.get_loc('Element')].Items)
Nb = len(IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items)
Ns = len(IndexTable.Classification[IndexTable.index.get_loc('Size')].Items)
Nh = len(IndexTable.Classification[IndexTable.index.get_loc('Recycling_Process')].Items)
NS = len(IndexTable.Classification[IndexTable.index.get_loc('EV_penetration_scenario')].Items)
Na = len(IndexTable.Classification[IndexTable.index.get_loc('Chemistry_Scenarios')].Items)
Nz = len(IndexTable.Classification[IndexTable.index.get_loc('Stock_Scenarios')].Items)
NR = len(IndexTable.Classification[IndexTable.index.get_loc('Reuse_Scenarios')].Items)
NE = len(IndexTable.Classification[IndexTable.index.get_loc('Energy_Storage_Scenarios')].Items)

INFO (<ipython-input-1-fb7cf16ecaa8> <<module>>): ### 1. - Initialize.
INFO (<ipython-input-1-fb7cf16ecaa8> <<module>>): Read and parse config table, including the model index table, from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read parameter list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read process list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model run control from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model output control from model config sheet.
INFO (ODYM_Functions.py <ParseClassificationFile_Main>): End of file or formatting error while reading the classification file in column 15. Check if all classifications are present. If yes, you are good to go!
INFO (<ipython-input-1-fb7cf16ecaa8> <<module>>): Define model classifications and select items for model classifications according to information provided by config file.
INFO (<ipython-input-1-fb7cf16ecaa8> <<module>

Vehicle stock model for European fleet


In [2]:
# Defining path to raw data
data_path = os.path.join(os.getcwd(), 'data', 'raw_data')

In [3]:
# Importing data
data = pd.read_excel(data_path+'/material_content.xlsx', sheet_name='kg per pack')
data.head()

,Size,DT,Element,LFP,NCA,NCM111,NCM523,NCM622,NCM622-Graphite (Si),NCM811-Graphite (Si),NCM955-Graphite (Si),Li-Sulphur,Li-Air,Li metals in Li-Sulphur,Li and Ni metals in Li-Air
0,Small,BEV,Li,3.115635,3.355852,4.632768,4.470037,3.895368,3.941125,3.345143,3.264520,0.288622,0.234142,5.43586,3.373528
1,Small,BEV,Ni,0.000000,22.166721,11.570129,16.756149,17.501836,17.740396,20.045448,21.998514,0.000000,0.000000,0.00000,4.559533
2,Small,BEV,Co,0.000000,4.171835,11.613485,6.727575,5.855806,5.935624,2.515070,1.226719,0.000000,0.000000,0.00000,0.000000
3,Small,BEV,Mn,0.000000,0.000000,10.827166,9.408103,5.459325,5.533738,2.344781,1.143661,0.000000,0.000000,0.00000,0.000000
4,Small,BEV,Al,44.111012,33.413643,36.670829,36.161843,34.657908,33.481218,31.684776,31.726733,24.626441,19.830296,0.00000,0.000000


In [4]:
# Bring data into desired format
data = data.melt(['DT', 'Size', 'Element'])
data.rename(columns= {'variable': 'Chemistry'}, inplace=True)
data.head()

,DT,Size,Element,Chemistry,value
0,BEV,Small,Li,LFP,3.115635
1,BEV,Small,Ni,LFP,0.000000
2,BEV,Small,Co,LFP,0.000000
3,BEV,Small,Mn,LFP,0.000000
4,BEV,Small,Al,LFP,44.111012


In [5]:
# Define empty array
MatArray = np.zeros((Ng,Ns,Nb,Ne))

In [6]:
# Materials to be considered
IndexTable.Classification[IndexTable.index.get_loc('Element')].Items

['Li', 'Graphite', 'P', 'Mn', 'Co', 'Ni']

In [7]:
for m in range(0, len(data.value.values)):
    try:
        DTPosition = IndexTable.Classification[IndexTable.index.get_loc('Drive_train')].Items.index(data.DT.iloc[m])
        SizePosition= IndexTable.Classification[IndexTable.index.get_loc('Size')].Items.index(data.Size.iloc[m])
        ChemistryPosition= IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items.index(data.Chemistry.iloc[m])
        ElementPosition= IndexTable.Classification[IndexTable.index.get_loc('Element')].Items.index(data.Element.iloc[m])
        MatArray[DTPosition, SizePosition, ChemistryPosition, ElementPosition] = data.value.iloc[m]
    except ValueError: # This is just to ignore elements that are not included in the model
        pass

In [8]:
results_path = os.path.join(os.getcwd(), 'data', 'scenario_data')
np.save(results_path+'/material_content', MatArray)